In [ ]:
# check gpu availability
import torch
torch.cuda.is_available()

In [ ]:
!nvidia-smi

In [ ]:
# fetch and extract data
!wget -c https://pcai056.informatik.uni-leipzig.de/downloads/corpora/deu-de_web-wrt_2019_30M.tar.gz 
!tar -xvzf /content/deu-de_web-wrt_2019_3M.tar.gz

In [ ]:
%%time 
# train tokenizer
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


paths = [str(x) for x in Path(".").glob("**/*.txt")]
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files='deu-de_web-wrt_2019_3M/deu-de_web-wrt_2019_3M-sentences.txt', vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [4]:
!mkdir kleinBERT
tokenizer.save_model("kleinBERT")

hello
CPU times: user 1.97 ms, sys: 3.11 ms, total: 5.08 ms
Wall time: 5.36 ms


In [ ]:
!head kleinBERT/merges.txt
!head kleinBERT/vocab.json

In [ ]:
#test tofrom tokenizers.implementations import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer(
    "./kleinBERT/vocab.json",
    "./kleinBERT/merges.txt",
)kenizer


In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
stupidstring = "Ich bin Marc und Ich liebe Kartoffeln."
tokenizer.encode(stupidstring)

In [ ]:
tokenizer.encode("Ich bin Jean-Phillipe und Ich liebe Kartoffeln.").tokens

In [ ]:
%%time
# train
from transformers import RobertaConfig, RobertaTokenizerFast, RobertaForMaskedLM
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("./kleinBERT", max_len=512)

In [ ]:
model = RobertaForMaskedLM(config=config)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./kleinBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=468750,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

trainer.train(model_path="kleinBERT")

In [ ]:
%%time
# compress and dl model binaries
!zip -r /content/kleinBERT.zip /content/kleinBERT/
from google.colab import files
files.download("/content/kleinBERT.zip")